In [13]:
20

20

In [13]:
20

20

In [1]:
from federatedscope.register import register_data
from federatedscope.register import register_trainer
from federatedscope.register import register_metric
from federatedscope.register import register_model




### Register model

In [2]:
#from federatedscope.contrib.model.mnist_model import call_my_net
#register_model("mynet", call_my_net)

### Register trainer

In [3]:

from federatedscope.contrib.trainer.laplacian_trainer import call_laplacian_trainer

register_trainer('laplacian_trainer', call_laplacian_trainer)

/home/michael/Master-Thesis/CKIM_Competition/venv/lib/python3.9/site-packages/torch_geometric/graphgym/imports.py:14: UserWarning: Please install 'pytorch_lightning' for using the GraphGym experiment manager via 'pip install pytorch_lightning'
  warnings.warn("Please install 'pytorch_lightning' for using the GraphGym "
/home/michael/Master-Thesis/CKIM_Competition/venv/lib/python3.9/site-packages/torch_geometric/graphgym/logger.py:23: UserWarning: Please install 'pytorch_lightning' for using the GraphGym experiment manager via 'pip install pytorch_lightning'
  warnings.warn("Please install 'pytorch_lightning' for using the GraphGym "


### Register metric

### Create the config

#### Set data, model, trainer and metric

In [4]:
from federatedscope.core.configs.config import global_cfg, CN
cfg = global_cfg.clone()

cfg.merge_from_file("scripts/B-FHTL_exp_scripts/Graph-DT/fedavg.yaml")
cfg.data.save_dir = 'test_dir'
# cfg.data.type = 'cikm_cup'
#cfg.data.root = 'data'
#cfg.data.shuffle=True
#cfg.data.transform = [['ToTensor'], ['Normalize', {'mean': [0.], 'std': [1]}]]
#cfg.model.type = 'gin'
#cfg.model.out_channels = 10
#cfg.model.hidden = 64
#cfg.model.task='graph'
#cfg.model.dropout = 0.5
#cfg.personalization.local_param = ['encoder_atom', 'encoder', 'clf']#['node_encoder', 'clf']
#cfg.train.batch_or_epoch = "epoch"
cfg.trainer.type = 'laplacian_trainer'
cfg.data.batch_size = 64
# cfg.eval.metric = ['mymetric']
cfg.params = CN()
cfg.params.alpha=0.1
cfg.params.csd_importance= 1e2
cfg.params.eps=1e-20
cfg.params.p=0.
cfg.params.lam = 1


#### configure other options

In [5]:
#cfg.use_gpu = True 0.05 [0.02-0.03] 0.04-0.06
#cfg.best_res_update_round_wise_key = "test_loss"

#cfg.federate.mode = 'standalone'

cfg.federate.method = \
    f'Laplacian_test'

#cfg.federate.local_update_steps = 20000000
cfg.personalization.local_update_steps = 20000000
#cfg.finetune.local_update_steps = 20000000
#cfg.train.local_update_steps = 1

cfg.federate.total_round_num = 2000
cfg.federate.client_num = 1
cfg.early_stop.patience = 100
#cfg.train.optimizer.lr = 0.001
#cfg.train.optimizer.weight_decay = 0.0005
#cfg.grad.grad_clip = 2.0
cfg.criterion.type = 'CrossEntropyLoss'
#cfg.seed = 123
cfg.eval.freq = 1
cfg.eval.metrics = ['imp_ratio']
cfg.eval.report = ['avg']
cfg.eval.best_res_update_round_wise_key = 'val_imp_ratio'
cfg.eval.count_flops = False


In [6]:
import torch
#torch.manual_seed(0)
#torch.backends.cudnn.benchmark = False
#torch.use_deterministic_algorithms(F/home/michael/Desktop/backup files/FederatedScope/data/CIKM22Competitionalse)
#import random
#random.seed(0)
#import numpy as np
#np.random.seed(0)

In [7]:
from yacs.config import CfgNode
client_cfg_file = "scripts/B-FHTL_exp_scripts/Graph-DT/cfg_per_client_theirs.yaml"# "scripts/B-FHTL_exp_scripts/Graph-DT/cfg_per_client_ours_lr_local_steps.yaml"
client_cfg = CfgNode.load_cfg(open(client_cfg_file,
                                       'r')) if client_cfg_file else None


### Start the FL prosess

In [8]:
from federatedscope.core.auxiliaries.data_builder import get_data
from federatedscope.core.auxiliaries.utils import setup_seed, update_logger
from federatedscope.core.fed_runner import FedRunner
from federatedscope.core.auxiliaries.worker_builder import get_server_cls, get_client_cls


2023-01-04 11:31:55,511 (trainer_builder:11)WARNING: No module named 'federatedscope.contrib.optimizer' in `federatedscope.contrib.trainer`, some modules are not available.


In [9]:
from federatedscope.contrib.workers.laplacian_with_domain_separation_MI_client import LaplacianDomainSeparationMIClient
from federatedscope.contrib.workers.laplacian_server_dom_sep import LaplacianServerDomSep

setup_seed(cfg.seed)
update_logger(cfg)
data, modified_cfg = get_data(cfg)
cfg.merge_from_other_cfg(modified_cfg)


2023-01-04 11:31:55,674 (utils:129)INFO: the current machine is at 127.0.1.1
2023-01-04 11:31:55,675 (utils:131)INFO: the current dir is /home/michael/Master-Thesis/CKIM_Competition
2023-01-04 11:31:55,676 (utils:132)INFO: the output dir is exp/test_dir/Laplacian_test_gin_on_graph-dt_lr0.1_lstep1_/sub_exp_20230104113155


In [10]:
from federatedscope.contrib.workers.laplacian_client import LaplacianClient
from federatedscope.contrib.workers.laplacian_server import LaplacianServer

Fed_runner = FedRunner(data=data,
                       server_class=LaplacianServer,
                       client_class=LaplacianClient,
                       config=cfg.clone(),
                       client_config=client_cfg)
Fed_runner.run()

/home/michael/Master-Thesis/CKIM_Competition/venv/lib/python3.9/site-packages/torch/nn/init.py:405: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


server params: 
encoder_atom.atom_embedding_list.0.weight


2023-01-04 11:31:57,819 (fed_runner:249)INFO: Server #0 has been set up ... 
2023-01-04 11:31:57,832 (config:261)INFO: the used configs are: 
asyn:
  min_received_num: 1
  min_received_rate: -1.0
  timeout: 0
  use: True
attack:
  alpha_TV: 0.001
  alpha_prop_loss: 0
  attack_method: 
  attacker_id: -1
  classifier_PIA: randomforest
  info_diff_type: l2
  inject_round: 0
  max_ite: 400
  reconstruct_lr: 0.01
  reconstruct_optim: Adam
  target_label_ind: -1
backend: torch
cfg_file: 
criterion:
  type: MSELoss
data:
  args: []
  batch_size: 64
  cSBM_phi: [0.5, 0.5, 0.5]
  consistent_label_distribution: False
  drop_last: False
  graphsaint:
    num_steps: 30
    walk_length: 2
  loader: 
  num_workers: 0
  pre_transform: []
  quadratic:
    dim: 1
    max_curv: 12.5
    min_curv: 0.02
  root: data/
  save_dir: test_dir
  server_holds_all: False
  shuffle: True
  sizes: [10, 5]
  splits: [0.8, 0.1, 0.1]
  splitter: ooxx
  splitter_args: []
  subsample: 1.0
  target_transform: []
  transf

encoder_atom.atom_embedding_list.1.weight
encoder_atom.atom_embedding_list.2.weight
encoder_atom.atom_embedding_list.3.weight
encoder_atom.atom_embedding_list.4.weight
encoder_atom.atom_embedding_list.5.weight
encoder_atom.atom_embedding_list.6.weight
encoder_atom.atom_embedding_list.7.weight
encoder_atom.atom_embedding_list.8.weight
encoder_atom.atom_embedding_list.9.weight
encoder_atom.atom_embedding_list.10.weight
encoder_atom.atom_embedding_list.11.weight
encoder_atom.atom_embedding_list.12.weight
encoder_atom.atom_embedding_list.13.weight
encoder_atom.atom_embedding_list.14.weight
encoder_atom.atom_embedding_list.15.weight
encoder_atom.atom_embedding_list.16.weight
encoder_atom.atom_embedding_list.17.weight
encoder_atom.atom_embedding_list.18.weight
encoder_atom.atom_embedding_list.19.weight
encoder_atom.atom_embedding_list.20.weight
encoder_atom.atom_embedding_list.21.weight
encoder.weight
encoder.bias
gnn.convs.0.nn.linears.0.weight
gnn.convs.0.nn.linears.0.bias
gnn.convs.0.nn.l

2023-01-04 11:31:58,596 (laplacian_client:118)INFO: {'Role': 'Client #1', 'Round': 0, 'Results_raw': {'train_loss': 9740.750422, 'train_avg_loss': 10.811044, 'train_total': 901, 'train_imp_ratio': -1069.646623}}
2023-01-04 11:31:58,616 (laplacian_server:160)INFO: Server #0: Starting evaluation at the end of round 0.


...

2023-01-04 11:32:00,837 (laplacian_client:118)INFO: {'Role': 'Client #1', 'Round': 6, 'Results_raw': {'train_loss': 2971.571701, 'train_avg_loss': 3.298082, 'train_total': 901, 'train_imp_ratio': -256.819392}}
2023-01-04 11:32:00,840 (server:480)INFO: {'Role': 'Server #', 'Round': 6, 'Results_avg': {'test_loss': 449.515377, 'test_avg_loss': 3.978012, 'test_total': 113.0, 'test_imp_ratio': -330.381029, 'val_loss': 364.157959, 'val_avg_loss': 3.222637, 'val_total': 113.0, 'val_imp_ratio': -248.657005}}
2023-01-04 11:32:00,842 (monitor:513)INFO: current_best=-147.03776, should_save=False
2023-01-04 11:32:00,843 (monitor:513)INFO: current_best=-147.03776, should_save=False
2023-01-04 11:32:00,868 (laplacian_server:160)INFO: Server #0: Starting evaluation at the end of round 6.
2023-01-04 11:32:00,871 (laplacian_server:167)INFO: ----------- Starting a new training round (Round #7) -------------


Time for omega computation: 3.2196044921875
Time for whole backward: 7.2917938232421875
Time for omega computation: 2.276182174682617
Time for whole backward: 7.198572158813477
Time for omega computation: 2.8624534606933594
Time for whole backward: 7.991552352905273
Time for omega computation: 2.5177001953125
Time for whole backward: 7.121801376342773
Time for omega computation: 3.0219554901123047
Time for whole backward: 10.038137435913086
Time for omega computation: 2.5620460510253906
Time for whole backward: 5.945920944213867
Time for omega computation: 2.223491668701172
Time for whole backward: 7.7571868896484375
Time for omega computation: 2.046823501586914
Time for whole backward: 6.5155029296875
Time for omega computation: 2.3345947265625
Time for whole backward: 6.781578063964844
Time for omega computation: 2.800464630126953
Time for whole backward: 7.284402847290039
Time for aggregation 1: 10.048627853393555
Time for aggregation 2: 20.210981369018555


2023-01-04 11:32:00,931 (client:410)INFO: {'Role': 'Client #1', 'Round': 7, 'Results_raw': {'test_loss': 341.710232, 'test_avg_loss': 3.023984, 'test_total': 113, 'test_imp_ratio': -227.164846, 'val_loss': 257.693041, 'val_avg_loss': 2.280469, 'val_total': 113, 'val_imp_ratio': -146.723919}}
2023-01-04 11:32:00,932 (monitor:513)INFO: current_best=-146.723919, should_save=True
2023-01-04 11:32:00,933 (client:431)INFO: Client: #1, val_imp_ratio: -146.723919. model saved at exp/test_dir/Laplacian_test_gin_on_graph-dt_lr0.1_lstep1_/sub_exp_20230104113155/model1.pth


Time for omega computation: 2.6853084564208984
Time for whole backward: 6.927967071533203
Time for omega computation: 2.581357955932617
Time for whole backward: 7.139444351196289
Time for omega computation: 3.3721923828125
Time for whole backward: 10.145425796508789
Time for omega computation: 1.9075870513916016
Time for whole backward: 5.583524703979492
Time for omega computation: 2.7589797973632812
Time for whole backward: 7.19761848449707
Time for omega computation: 2.5947093963623047
Time for whole backward: 6.904363632202148
Time for omega computation: 2.4056434631347656
Time for whole backward: 6.79779052734375
Time for omega computation: 3.412485122680664
Time for whole backward: 7.915258407592773
Time for omega computation: 2.414226531982422
Time for whole backward: 6.92296028137207


KeyboardInterrupt: 

client 0 times: 0.346s
bei normal fedavg: 0.15s
mit MINE: 1.7 Sekunden


+

In [11]:
777-431


346

In [13]:
data

{1: {'train': <torch_geometric.loader.dataloader.DataLoader at 0x7f4ebc607190>,
  'val': <torch_geometric.loader.dataloader.DataLoader at 0x7f4de953c910>,
  'test': <torch_geometric.loader.dataloader.DataLoader at 0x7f4e247e3970>,
  'num_label': 0},
 2: {'train': <torch_geometric.loader.dataloader.DataLoader at 0x7f4ebc607190>,
  'val': <torch_geometric.loader.dataloader.DataLoader at 0x7f4de953c910>,
  'test': <torch_geometric.loader.dataloader.DataLoader at 0x7f4e247e3970>,
  'num_label': 0}}

In [ ]:
import os

,,0After 10 rounds:

diff: 0, csd: 0:
server: -8
c1 t/val: 0.375 /0.37
c2 t/val: 0.678 /0.606
c3 t/val: 0.699 /0.608
c4 t/val: 0.599 /0.537

diff: 0, csd: 0:
server: -4.5
c1 t/val: 0.372879 /0.328
c2 t/val: 0.678992 /0.605809
c3 t/val: 0.70166 /0.608609
c4 t/val: 0.597455 /0.534253



diff: 10, csd: 0:
server: -35
c1 t/val: 0.482/0.52
c2 t/val: 0.694/0.65
c3 t/val: 0.724/0.65
c4 t/val: 0.674/0.575

diff: 1, csd: 0:
server: -9.6
c1 t/val: 0.378194/0.319349
c2 t/val: 0.67461/0.609163
c3 t/val: 0.705559/0.615019
c4 t/val: 0.605581/0.572086

diff: 0, csd: 1e4:
server: -18
c1 t/val: 0.280962/0.23074
c2 t/val: 0.682652/0.603175
c3 t/val: 0.699094/0.67027
c4 t/val: 0.594985/0.609611

diff: 0 csd: 1e8
server: -18
c1 t/val: 0.2606/0.348(0.197)
c2 t/val: 0.653/0.0.602
c3 t/val: 0.691/0.597
c4 t/val: 0.581/0.629

diff: 0 csd: 1e8
server: -12.7
c1 t/val: 0.262647/0.249875
c2 t/val: 0.649273/0.0.600435
c3 t/val: 0.696132/0.615158
c4 t/val: 0.586597/0.656992


diff: 0.1 csd: 0
server: -11
c1 t/val: 0.2606/0.332837
c2 t/val: 0.678326/0.0.610099
c3 t/val: 0.704083/0.616179
c4 t/val: 0.604177/0.559658




In [4]:
os.environ['CUBLAS_WORKSPACE_CONFIG']

NameError: name 'os' is not defined